# SVM Classifier For Feelings

we start first with building a function that preprocess data, and should be able to understand negation. the objectif is to transform a sentence like "i don't like banana" to [I, don't, not-like, banana]

In [19]:
text="i don't like what are you saying"

def add_not(text):
    liste=["don't","wasn't","weren't","didn't"]
    mots=text.split(' ')
    for i in range (len(mots)):
        if mots[i] in liste:
            mots[i+1]='not'+"-" +mots[i+1]
    return mots
print(add_not(text))

['i', "don't", 'not-like', 'what', 'are', 'you', 'saying']


In [27]:
import nltk
import os

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

# Download the list of stop words if you haven't already
nltk.download('stopwords')
nltk.download('punkt')

def preprocess_text(text):
    # Tokenize the text into words
    words =add_not(text) 

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word not in stop_words]

    # Perform stemming using Porter Stemmer
    stemmer = PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in filtered_words]


    return  " ".join(stemmed_words)

def load_training_set():
    training= []
    y_train=[]
    folder_train_path=r"C:\Users\Hp\Desktop\mes projets\NLP and  text analysis\mini_dataset_feeling_analysis\train"
    folder_train_path_positive=os.path.join(folder_train_path,'positive')
    text_names=os.listdir(folder_train_path_positive)
    for text in text_names:
        text_path=os.path.join(folder_train_path_positive,text)
        with open(text_path, 'r') as file:
            contenu = file.read()
            training.append(preprocess_text(contenu))
            y_train.append('positive')
    folder_train_path_negative=os.path.join(folder_train_path,'negative')
    text_names=os.listdir(folder_train_path_negative)
    for text in text_names:
        text_path=os.path.join(folder_train_path_negative,text)
        with open(text_path, 'r') as file:
            contenu = file.read()
            training.append(preprocess_text(contenu))
            y_train.append("negative")
    return training,y_train

training,y_train= load_training_set()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [28]:
print(training[:2])

['bromwel high cartoon comedy. it ran time program school life, "teachers". my 35 year teach profess lead believ bromwel high\' satir much closer realiti "teachers". the scrambl surviv financially, insight student see right pathet teachers\' pomp, petti whole situation, remind school i knew students. when i saw episod student repeatedli tri burn school, i immedi recal ......... .......... high. a classic line: inspector: i\'m sack one teachers. student: welcom bromwel high. i expect mani adult age think bromwel high far fetched. what piti isn\'t!', "scott bartlett' 'offon' nine minut pure craziness. it full-front assault psychedelic, pulsating, epilepsy-induc flash light colours, first true merg film video avante-gard cinema. there' stori speak of, bartlett use imag natur â– particularli human face form â– provok sequenc emot reactions, integr biolog phenomena highly-industri form modern technology. in sense, film repres merg human tools, machinery, technology. thi theme connect loos s

In [35]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # You can adjust max_features as needed
X_train = tfidf_vectorizer.fit_transform(training).toarray()

In [36]:
from sklearn.svm import SVC
# Step 4: Create and train the SVM model
svm_model = SVC(kernel='linear')  # You can use different kernels like 'rbf', 'poly', etc.
svm_model.fit(X_train, y_train)

SVC(kernel='linear')

In [37]:
def load_test_set():
    test = []
    y_test=[]
    folder_test_path=r"C:\Users\Hp\Desktop\mes projets\NLP and  text analysis\mini_dataset_feeling_analysis\test"
    folder_test_path_positive=os.path.join(folder_test_path,'positive')
    text_names=os.listdir(folder_test_path_positive)
    for text in text_names:
        text_path=os.path.join(folder_test_path_positive,text)
        with open(text_path, 'r') as file:
            contenu = file.read()
            test.append(preprocess_text(contenu))
            y_test.append("positive")
    folder_test_path_negative=os.path.join(folder_test_path,'negative')
    text_names=os.listdir(folder_test_path_negative)
    for text in text_names:
        text_path=os.path.join(folder_test_path_negative,text)
        with open(text_path, 'r') as file:
            contenu = file.read()
            test.append(preprocess_text(contenu))
            y_test.append("negative")
    return test,y_test

test,y_test = load_test_set()

In [38]:
X_test = tfidf_vectorizer.transform(test).toarray()

In [39]:
y_pred = svm_model.predict(X_test)

# Step 6: Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.696078431372549


In [48]:
from sklearn.model_selection import GridSearchCV
kernel_list=['rbf','poly',"linear"]
grid_params={"kernel":kernel_list}
grid=GridSearchCV(SVC(),grid_params,
                  cv=5,
                  scoring='accuracy')
grid.fit(X_train,y_train)


GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'kernel': ['rbf', 'poly', 'linear']},
             scoring='accuracy')

In [49]:
best_params = grid.best_params_
print("Best Hyperparameters:", best_params)

# Get the best model found by GridSearchCV
best_model = grid.best_estimator_

# Make predictions using the best model
y_pred = best_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Best Accuracy:", accuracy)

Best Hyperparameters: {'kernel': 'linear'}
Best Accuracy: 0.696078431372549
